In [1]:
import numpy as np
import tensorflow as tf
import graph_nets as gn
import sonnet as snt
import matplotlib.pyplot as plt
import math
import time
import datetime
import gc
import os
import sys
import pickle

#from pysr import pysr

from simulator import read_orbits, base_classes
from data.solar_system_names import *
from ml_model_long import *

sys.modules['base_classes'] = base_classes
print('Started')

tf.config.list_physical_devices('CPU')
tf.config.run_functions_eagerly(False)


Metal device set to: Apple M1
Started


2022-05-27 16:38:35.246497: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-27 16:38:35.246592: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Global Constants

In [2]:
start_time = time.time()

# Global constants
AU = 149.6e6 * 1000 # Astronomical Unit in meters.
DAY = 24*3600. # Day in seconds
YEAR = 365.25*DAY #Year
delta_time = (0.5/24.) # 2 hours
MSUN = 1.9885e+30
MEARTH = 5.9724e+24
G = 6.67428e-11/AU**3*MSUN*DAY**2

Training variables

In [3]:
# Training variables
patience = 20 # For early stopping
noise_level = 0.01 # Standard deviation of Gaussian noise for randomly perturbing input data
num_epochs = 10000 # Number of training epochs. Set to large number
num_time_steps_tr = 8000  # Number of time steps for training (~27 years).
# One time step is 30 minutes
# An orbit for saturn is 129110 steps
num_time_steps_val = 80 # Using few to speed up calculations

Read the data

In [4]:
def force_newton(x, m1, m2):
    return G*m1*m2/np.linalg.norm(x, axis = -1, keepdims=True)**3.*x

In [5]:
def read_data(num_time_steps_tr, num_time_steps_val):
    """
    Read the data
    :param num_time_steps_tr: Size of training set
    :param num_time_steps_val: Size of validation set
    :return: training data, validation data, and a Star System object
    containing three dimensions: Time, body, and a length 6 dimension with
    x and v
    """

    # Read the file
    filename = './data/full.pkl'
    filehandler = open(filename, 'rb')
    system = pickle.load(filehandler)

    # Extract the position and velocity
    x = system.get_positions()
    v = system.get_velocities()

    # Concatenate them into a numpy array
    # Data has three dimensions: Time, body, and a length 6 dimension with x and v
    data = np.concatenate([x, v], axis=-1)

    # Convert velocity into acceleration
    A = data[1:, :, 3:] - data[:-1, :, 3:]
    data[:-1, :, 3:] = A / delta_time
    data = data[:-1]

    # Eliminate unused data
    data = data[:(num_time_steps_tr + num_time_steps_val)]

    # Split into training and validation
    data_tr = data[:num_time_steps_tr]
    data_val = data[num_time_steps_tr:]

    # Shuffle the data
    np.random.shuffle(data_tr)
    np.random.shuffle(data_val)

    return data_tr, data_val, system


def format_data(data_tr, data_val, system):
    """
    Convert the data into normalized tensorflow data objects that we can
    use for training
    """

    num_time_steps_tr = len(data_tr)

    # Do not change this
    batch_size_tr = 8
    num_batches = num_time_steps_tr // batch_size_tr

    nedges = system.numEdges

    # Create empty arrays for the distances for training and validation
    D_tr = np.empty([len(data_tr), nedges, 3])
    D_val = np.empty([len(data_val), nedges, 3])

    k = 0
    # Create empty lists for the senders and receivers that will be used for
    # the edges of the graph
    senders, receivers = [], []
    for i in range(system.numPlanets):
        for j in range(system.numPlanets):
            if i > j:
                # For every pair of objects, assign a distance
                D_tr[:, k, :] = data_tr[:, j, :3] - data_tr[:, i, :3]
                D_val[:, k, :] = data_val[:, j, :3] - data_val[:, i, :3]

                k += 1
                # Add sender and receiver index
                receivers.append(i)
                senders.append(j)

    # Accelerations
    A_tr = data_tr[:, :, 3:]
    A_val = data_val[:, :, 3:]
    # Normalization of the accelerations
    A_norm = G*1e-10 #np.std(A_tr)

    # Flatten the arrays
    D_tr = np.reshape(D_tr, [-1, 3])
    D_val = np.reshape(D_val, [1, -1, 3])

    A_tr = np.reshape(A_tr / A_norm, [-1, 3])
    A_val = np.reshape(A_val / A_norm, [1, -1, 3])

    # Convert them to tensors
    D_tr = tf.convert_to_tensor(D_tr, dtype="float32")
    A_tr = tf.convert_to_tensor(A_tr, dtype="float32")

    D_val = tf.convert_to_tensor(D_val, dtype="float32")
    A_val = tf.convert_to_tensor(A_val, dtype="float32")

    # Split the training arrays into batches
    D_tr_batches = tf.split(D_tr, num_batches)
    A_tr_batches = tf.split(A_tr, num_batches)

    # Convert into tensorflow dataset
    train_ds = tf.data.Dataset.from_tensor_slices(
        (D_tr_batches, A_tr_batches))

    test_ds = tf.data.Dataset.from_tensor_slices(
        (D_val, A_val))

    # Create a normalization layer
    norm_layer = Normalize_gn(cartesian_to_spherical_coordinates(D_tr))
    return train_ds, test_ds, norm_layer, senders, receivers

In [6]:
def check_data(data_val, system):
    """
    Convert the data into normalized tensorflow data objects that we can
    use for training
    """
    nedges = system.numEdges

    # Create empty arrays for the distances for training and validation
    D_val = np.empty([10, nedges, 3])

    k = 0
    # Create empty lists for the senders and receivers that will be used for
    # the edges of the graph
    senders, receivers = [], []
    for i in range(system.numPlanets):
        for j in range(system.numPlanets):
            if i > j:
                # For every pair of objects, assign a distance
                D_val[:, k, :] = data_val[:10, j, :3] - data_val[:10, i, :3]

                k += 1

    A_val = data_val[:10, :, 3:]
    
    return D_val, A_val

In [7]:
#d, a = check_data(data_val, system)

In [8]:
#G * masses[0] * masses[2] * d[:,1] / d[:,1]**3 / masses[2] / (G*1e10)

In [9]:
data_tr, data_val, system = read_data(num_time_steps_tr, num_time_steps_val)
train_ds, test_ds, norm_layer, senders, receivers = format_data(data_tr, data_val, system)

In [10]:
checkpoint_filepath = './saved_models/full'

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            verbose = 1,
                                            patience=patience,
                                            #baseline = 0.1,
                                            restore_best_weights=False)
# Restore best weights not working, but found way around using checkpoint

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                 save_weights_only=False,
                                                save_best_only=True,
                                                 verbose=0)

model = LearnForces(system.numPlanets, senders, receivers, norm_layer,
                        noise_level=noise_level, masses=system.get_masses())

#model.compile(run_eagerly=True)
model.compile()

In [11]:
model.evaluate(test_ds)

2022-05-27 16:39:04.961735: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-27 16:39:04.962218: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-27 16:39:05.650037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step - loss: inf


inf

In [ ]:
model.fit(train_ds, 
          epochs = num_epochs, 
          verbose=1, 
          callbacks=[early_stopping, checkpoint], 
          validation_data=test_ds
         )


Epoch 1/10000


2022-05-27 16:39:09.827532: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-05-27 16:39:12.709097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 984/1000 [============================>.] - ETA: 1s - loss: inf

In [ ]:
model.save("./saved_models/full")

In [ ]:
model2 = LearnForces(system.numPlanets, senders, receivers, norm_layer,
                        noise_level=noise_level, masses=system.get_masses())

model2.compile()
model2.evaluate(test_ds)
#model.compile(run_eagerly=True)

In [ ]:
model2.load_weights("./saved_models/tedt2")

In [ ]:
model2.evaluate(test_ds)

In [ ]:
for mass, true_mass, name in zip(model.logm_planets.numpy(), system.get_masses(), system.get_names()):
    print(name, np.round(np.log10(true_mass + 1e-22) + 10, 2), np.round(mass, 2))


In [ ]:
model2.evaluate(test_ds)

In [ ]:
model.load_weights(checkpoint_filepath)

model.evaluate(test_ds)

In [ ]:
tf.keras.models.load_model("./saved_models/orbits/reduced_weights.ckpt.data-00000-of-00001")

In [ ]:
j=0
learned_masses = model.logm_planets.numpy() - model.logm_planets.numpy()[j]
for i in range(nplanets):
    print(f'{names[i]}, {np.log10(masses[i]/masses[j]):.2f}, {learned_masses[i]:.2f}, {abs(np.log10(masses[i]/masses[j]) -learned_masses[i])/abs(np.log10(masses[i])):.2f}')

train_time = time.time()

# Evaluate on validation data

In [ ]:
ap ,fp = model(D_val_flat[0], extract = True)

In [ ]:
F_val_new = np.empty([len(data_val), nedges, 3])
k=0
for i in range(nplanets):
    for j in range(nplanets):
        if i > j:
            d_val = data_val[:,j,:3] - data_val[:,i,:3]
            F_val_new[:,k,:] = force_newton(d_val, 10**learned_masses[i], 10**learned_masses[j]) #cartesian_to_spherical_coordinates(d_val)
            k+=1

In [ ]:
nrows = math.ceil(nplanets/4)
fig, ax = plt.subplots(nrows, 4, figsize = (16, 4*nrows))
for i in range(nplanets):
    ax[i//4, i%4].set_title(names[i], fontsize = 16)
    ax[i//4, i%4].plot(ap[:,i,0], ap[:,i,1], '.', label = 'Learned')
    ax[i//4, i%4].plot(data_val[:,i,3]/A_norm, data_val[:,i,4]/A_norm, '.', label = 'Truth')

ax[0,0].legend()
#plt.savefig('/Users/Pablo/Desktop/full_learnedmasses.png')

In [ ]:
nrows = math.ceil(nedges/4)
fig, ax = plt.subplots(nrows, 4, figsize = (16, 4*nrows))
for i in range(nedges):
    ax[i//4, i%4].set_title(names_edges[i])
    #ax[i//4, i%4].plot(F_val_new[:,i,0], F_val_new[:,i,1], '.')
    #ax[i//4, i%4].plot(fp[:,i,0]*n1/n2, fp[:,i,1]*n1/n2, '.')
    ax[i//4, i%4].plot(fp[:,i,0], fp[:,i,1], '.')
#plt.savefig('/Users/Pablo/Desktop/forces_learnedmasses.png')

In [ ]:
nrows = math.ceil(nedges/4)
fig, ax = plt.subplots(nrows, 4, figsize = (16, 4*nrows))
for i in range(nedges):
    ax[i//4, i%4].set_title(names_edges[i])
    ax[i//4, i%4].plot(F_val[:,i,0]*1e8, F_val[:,i,1]*1e8, '.')
    #ax[i//4, i%4].plot(fp[:,i,0]*n1/n2, fp[:,i,1]*n1/n2, '.')
    #ax[i//4, i%4].plot(fp[:,i,0], fp[:,i,1], '.')
#plt.savefig('/Users/Pablo/Desktop/forces_learnedmasses.png')

In [ ]:
plot_time = time.time()

In [ ]:
X = np.zeros([nval, nedges,9])
X[:,:,2:5] = D_val_np
X[:,:,5] = np.linalg.norm(D_val_np, axis = -1)
X[:,:,6:] = fp
k=0
for i in range(nplanets):
    for j in range(nplanets):
        if i > j:
            X[:,k,0] = 10**(learned_masses[i])
            X[:,k,1] = 10**(learned_masses[j])
            #print(fp[0,k,0], X[0,k,0]*X[0,k,1]*X[0,k,2]/X[0,k,5]**3.)
            k+=1 
            
N = 500
ii = np.random.choice(nval, N, replace = False)
X = X[ii]

X[:,:,2:5], X[:,:,6:] = rotate_data(X[:,:,2:5], X[:,:,6:], uniform = False)

X = np.reshape(X, [N*nedges,9])

In [ ]:
for i in range(len(X)):
    r = np.random.rand()
    if r > 0.5:
        X[i,0], X[i,1]  = X[i,1], X[i,0]

In [ ]:
from sklearn.neighbors import KernelDensity
norm_F = np.linalg.norm(X[:,6:], axis = -1)
X_density = norm_F[:, None]
bin_width = np.max(X_density)/30
kde = KernelDensity(bandwidth=bin_width,
                    kernel='cosine') #Want finite cutoff so not too expensive!
kde.fit(X_density)

In [ ]:
grid = np.linspace(0.0, 1.5e-3, num=1000)
density_grid = np.exp(kde.score_samples(grid[:, None]))
grid_idx = np.argmax((X_density < grid[1:]) & (X_density > grid[:-1]), axis=1)
density_values = density_grid[grid_idx]
inv_density = 1 / density_values
inv_density /= inv_density.sum()
np.random.seed(1)
idx_for_pysr = np.random.choice(np.arange(len(X)), size=(1000,), p=inv_density)
X_pysr = X[idx_for_pysr]

In [ ]:
y = X_pysr[:,6] #F_x
#X[:, [0, 1]] = np.exp(X[:, [0, 1]])/1e23 #re-scale to prevent precision issues, since pysr uses 32-bit floats
y /= np.std(y)                                 #same as above

m_std = np.std(X_pysr[:,:2])
x_std = np.std(X_pysr[:,2:5])
X_pysr[:,:2]/= m_std
X_pysr[:,2:]/= x_std

kwargs = dict(populations=64, binary_operators="+ * - /".split(" "),
          unary_operators=[], temp_equation_file=True,
          progress=False, procs=4, annealing=False, maxsize=40, useFrequency=True,  
          variable_names = ['m0', 'm1', 'x', 'y', 'z', 'r'],
          optimizer_algorithm="BFGS",
          optimizer_iterations=10,
          optimize_probability=1.0,)
equations=pysr(X_pysr[:,:6], y, **kwargs)

In [ ]:
from pysr import best
best(equations)

In [ ]:
sr_time = time.time()

In [ ]:
print('Total time: ', str(datetime.timedelta(seconds=sr_time - start_time)))
print('---------------')
print('Time to set up data: ', str(datetime.timedelta(seconds=setup_time - start_time)))
print('Time for training: ', str(datetime.timedelta(seconds=train_time - setup_time)))
print('Time for plotting: ', str(datetime.timedelta(seconds=plot_time - train_time)))
print('Time for symbolic regression: ', str(datetime.timedelta(seconds=sr_time - plot_time)))

# TO DO:

### To finish the project
- Working!

### To clean the model
- Improve plotting

# Requirements
- The graphnets tensorflow 2 installation
- pysr
- matplotlib

# Debugging (ignore)

In [ ]:
from tensorflow.python.keras.utils import tf_utils

In [ ]:
loss_tr = 1

i = 0
while not np.isnan(loss_tr):
    model = LearnForces(nplanets, senders, receivers, norm_layer, noise_level = noise_level)
    model.compile(run_eagerly=True)

    model.fit(train_ds, 
              verbose = 2,
              epochs = 1,
              steps_per_epoch = 10,
              callbacks=[
                    tf.keras.callbacks.TerminateOnNaN()]
              #validation_data=test_ds
             )
    
    loss_tr = tf_utils.to_numpy_or_python_type(model.evaluate(ini_ds, verbose = 0))
    i+=1
    if i>1000: 
        print('SUCCESS!')
        break
    #print(loss_tr)




In [ ]:
loss_tr = np.NaN

while np.isnan(loss_tr):
    model = LearnForces(nplanets, senders, receivers, norm_layer, noise_level = noise_level)
    model.compile(run_eagerly=True)

    model.fit(train_ds, 
              verbose = 2,
              epochs = 1,
              steps_per_epoch = 10,
              callbacks=[
                    tf.keras.callbacks.TerminateOnNaN()]
              #validation_data=test_ds
             )
    
    loss_tr = tf_utils.to_numpy_or_python_type(model.evaluate(ini_ds, verbose = 0))
    #print(loss_tr)



In [ ]:
def convert(gm):
    return gm/6.67428e-11*1e9

In [ ]:
print(f"{convert(324858.592) :.2e}")